# **Classification du churn dans une entreprise de services numériques**

L'objectif est de prédire la résiliation des clients d'une entreprise de services téléphoniques et Internet à l'aide d'un modèle de Machine Learning.

- `CustomerID` : Un identifiant unique pour chaque client.
- `Zip Code` : Le code postal de la résidence principale du client.
- `Lat Long` : Un tuple représentant les coordonnées GPS du Zip Code.
- `Gender` : Le genre du client : Masculin, Féminin.
- `Senior Citizen` : Indique si le client a 65 ans ou plus : Oui, Non.
- `Partner` : Indique si le client a un partenaire : Oui, Non.
- `Dependents` : Indique si le client vit avec des personnes à charge : Oui, Non. Les personnes à charge peuvent être des enfants, parents, grands-parents, etc.
- `Tenure Months` : Indique le nombre total de mois que le client a passé avec l'entreprise à la fin du trimestre d'étude.
- `Phone Service` : Indique si le client est abonné à un service de téléphone à domicile avec l'entreprise : Oui, Non.
- `Multiple Lines` : Indique si le client est abonné à plusieurs lignes téléphoniques avec l'entreprise : Oui, Non.
- `Internet Service` : Indique si le client est abonné à un service Internet avec l'entreprise : Non, DSL, Fibre optique.
- `Online Security` : Indique si le client est abonné à un service supplémentaire de sécurité en ligne fourni par l'entreprise : Oui, Non.
- `Online Backup` : Indique si le client est abonné à un service supplémentaire de sauvegarde en ligne fourni par l'entreprise : Oui, Non.
- `Device Protection` : Indique si le client est abonné à un plan de protection supplémentaire pour son équipement Internet fourni par l'entreprise : Oui, Non.
- `Tech Support` : Indique si le client est abonné à un plan de support technique supplémentaire avec des temps d'attente réduits : Oui, Non.
- `Streaming TV` : Indique si le client utilise son service Internet pour diffuser des programmes télévisés via un fournisseur tiers : Oui, Non. L'entreprise ne facture pas de frais supplémentaires pour ce service.
- `Streaming Movies` : Indique si le client utilise son service Internet pour diffuser des films via un fournisseur tiers : Oui, Non. L'entreprise ne facture pas de frais supplémentaires pour ce service.
- `Contract` : Indique le type de contrat actuel du client : Mois par Mois, Un an, Deux ans.
- `Paperless Billing` : Indique si le client a opté pour la facturation sans papier : Oui, Non.
- `Payment Method` : Indique comment le client paye sa facture : Prélèvement bancaire, Carte de crédit, Chèque envoyé par courrier, Chèque automatique.
- `Monthly Charges` : Indique le montant total actuel mensuel des services de l'entreprise pour le client.
- `Total Charges` : Indique les frais totaux du client, calculés jusqu'à la fin du trimestre spécifié ci-dessus.
- `CLTV` : Valeur vie client (Customer Lifetime Value). Une CLTV prédite est calculée à l'aide de formules d'entreprise et de données existantes. Plus la valeur est élevée, plus le client est précieux. Les clients de grande valeur doivent être surveillés pour éviter leur départ.
- `Churn Value` : 1 = le client a quitté l'entreprise ce trimestre. 0 = le client est resté avec l'entreprise. Ceci est la variable à prédire.

In [1]:
import os
import joblib
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from sklearn.cluster import KMeans

In [2]:
# Retour au dossier racine
os.chdir("..")

## **2. Préparation des données**

In [3]:
# Chargement
df = pd.read_csv("./data/DatasetChurn.csv")

# Suppression des valeurs manquantes
df.dropna(inplace=True)

# Extraction de la latitude et de la longitude
df["Lat"] = df["Lat Long"].str.split(", ").str[0].astype(float)
df["Long"] = df["Lat Long"].str.split(", ").str[1].astype(float)

# Identification des nouveaux clients
df["Is New Client"] = (df["Tenure Months"] <= 48).astype(int)

# Séparation avec/sans Internet
df_internet = df.query("`Internet Service` != 'No'").copy()
df_no_internet = df.drop(df_internet.index).copy()

### **2.1 Avec Internet**

In [4]:
# Création des clusters de `Monthly Charges`, puis réorganisation des labels
# selon l'ordre croissant de leurs médianes pour garantir une progression logique
kmeans_internet = KMeans(n_clusters=20, random_state=42)
df_internet["Monthly Charges Group"] = kmeans_internet.fit_predict(
    df_internet[["Monthly Charges"]]
)

cluster_medians = df_internet.groupby("Monthly Charges Group")[
    "Monthly Charges"
].median()
sorted_clusters = cluster_medians.sort_values().index
clusters_mapping_internet = {old: new for new, old in enumerate(sorted_clusters)}
df_internet["Monthly Charges Group"] = df_internet["Monthly Charges Group"].map(
    clusters_mapping_internet
)

# Exportation du modèle de clustering de `Monthly Charges`
joblib.dump(kmeans_internet, "./artifacts/internet/kmeans_internet.pkl")
joblib.dump(
    clusters_mapping_internet, "./artifacts/internet/clusters_mapping_internet.pkl"
)

# Ajout de la colonne `Services Count` pour compter le nombre de services auxquels le client a souscrit
internet_services = [
    "Online Security",
    "Online Backup",
    "Device Protection",
    "Tech Support",
    "Streaming TV",
    "Streaming Movies",
]
df_internet["Services Count"] = (
    df_internet[internet_services]
    .replace({"No": "0", "Yes": "1"})
    .astype(int)
    .sum(axis=1)
)

# Vision d'ensemble
df_internet.nunique()

CustomerID               4569
Zip Code                 1638
Lat Long                 1638
Gender                      2
Senior Citizen              2
Partner                     2
Dependents                  2
Tenure Months              72
Phone Service               2
Multiple Lines              3
Internet Service            2
Online Security             2
Online Backup               2
Device Protection           2
Tech Support                2
Streaming TV                2
Streaming Movies            2
Contract                    3
Paperless Billing           2
Payment Method              4
Monthly Charges          1427
Total Charges            4435
Churn Value                 2
CLTV                     2798
Lat                      1638
Long                     1637
Is New Client               2
Monthly Charges Group      20
Services Count              7
dtype: int64

### **2.2. Sans Internet**

In [5]:
# Création des clusters de `Monthly Charges`, puis réorganisation des labels
# selon l'ordre croissant de leurs médianes pour garantir une progression logique
kmeans_no_internet = KMeans(n_clusters=2, random_state=42)
df_no_internet["Monthly Charges Group"] = kmeans_no_internet.fit_predict(
    df_no_internet[["Monthly Charges"]]
)

cluster_medians = df_no_internet.groupby("Monthly Charges Group")[
    "Monthly Charges"
].median()
sorted_clusters = cluster_medians.sort_values().index
clusters_mapping_no_internet = {old: new for new, old in enumerate(sorted_clusters)}
df_no_internet["Monthly Charges Group"] = df_no_internet["Monthly Charges Group"].map(
    clusters_mapping_no_internet
)

# Exportation du modèle de clustering de `Monthly Charges`
joblib.dump(kmeans_no_internet, "./artifacts/no_internet/kmeans_no_internet.pkl")
joblib.dump(
    clusters_mapping_no_internet,
    "./artifacts/no_internet/clusters_mapping_no_internet.pkl",
)

# Vision d'ensemble
df_no_internet.nunique()

CustomerID               1442
Zip Code                 1038
Lat Long                 1038
Gender                      2
Senior Citizen              2
Partner                     2
Dependents                  2
Tenure Months              72
Phone Service               1
Multiple Lines              2
Internet Service            1
Online Security             1
Online Backup               1
Device Protection           1
Tech Support                1
Streaming TV                1
Streaming Movies            1
Contract                    3
Paperless Billing           2
Payment Method              4
Monthly Charges           121
Total Charges            1332
Churn Value                 2
CLTV                     1218
Lat                      1038
Long                     1038
Is New Client               2
Monthly Charges Group       2
dtype: int64

## **3. Model**

In [6]:
param_grid = {
    "learning_rate": np.arange(0.01, 0.05, 0.01),  # taux d'apprentissage
    "scale_pos_weight": np.arange(
        3, 8, 1
    ),  # poids associé aux points positifs (minoritaires)
    "subsample": np.arange(0.8, 1, 1),  # proportion des lignes sélectionnées
}

### **3.1. Avec Internet**

- Nous entraînons d'abord un modèle de régression logistique en testant diverses combinaisons de variables.
- La métrique d'intérêt est le rappel; plus il sera élevé et plus on aura de chances d'anticiper les résiliations.

In [7]:
disc = [
    # "Gender",
    "Senior Citizen",
    "Partner",
    "Dependents",
    # "Phone Service",
    "Multiple Lines",
    "Internet Service",
    "Online Security",
    # "Online Backup",
    "Device Protection",
    "Tech Support",
    "Streaming TV",
    # "Streaming Movies",
    "Contract",
    "Paperless Billing",
    # "Payment Method",
    "Is New Client",
]

cont = [
    # "Tenure Months",
    "Monthly Charges",
    "Total Charges",
    "CLTV",
    "Lat",
    "Long",
    "Monthly Charges Group",
    "Services Count",
]

X_internet = df_internet.drop(columns="Churn Value")
y_internet = df_internet["Churn Value"]
X_train_internet, X_test_internet, y_train_internet, y_test_internet = train_test_split(
    X_internet, y_internet, test_size=0.2, random_state=42, stratify=y_internet
)

preprocessor_internet = ColumnTransformer(
    transformers=[
        ("disc", OneHotEncoder(drop="first", handle_unknown="ignore"), disc),
        ("cont", MinMaxScaler(), cont),
    ],
    remainder="drop",
)

X_train_internet_preprocessed = preprocessor_internet.fit_transform(X_train_internet)

clf_internet = LogisticRegression(random_state=42)
print(
    f"LogisticRegression : {cross_val_score(clf_internet, X_train_internet_preprocessed, y_train_internet, scoring='recall', cv=5).mean():.4f}"
)

LogisticRegression : 0.3155


In [8]:
clf_internet = XGBClassifier(random_state=42)
print(
    f"XGBClassifier : {cross_val_score(clf_internet, X_train_internet_preprocessed, y_train_internet, scoring='recall', cv=5).mean()}"
)

XGBClassifier : 0.36080302314596124


- Le modèle de régression logistique est un modèle assez simple qui a servi pour un premier benchmark, mais nous pensons qu'un modèle de type gradient boosting pourrait être mieux ajusté pour augmenter significativement le score.
- A la suite de plusieurs tests, on obtient les meilleures performances en supprimant les variables `Gender`, `Phone Service`, `Online Backup`, `Streaming Movies`, `Payment Method` et `Tenure Months`.
- Nous effectuons une recherche d'hyperparamètres avec validation-croisée à l'aide d'un `GridSearchCV` en optimisant le score f1 plutôt que le rappel.
- L'objectif est d'atteindre un équilibre entre précision et rappel, au lieu d'avoir un rappel trop élevé au détriment de la précision.

In [9]:
model_internet = GridSearchCV(clf_internet, param_grid, scoring="f1", cv=5)
model_internet.fit(X_train_internet_preprocessed, y_train_internet)

print("Paramètres :", model_internet.best_params_)
print("Score (f1) :", model_internet.best_score_)

# Exportation du pipeline de preprocessing et du modèle entraînés
joblib.dump(preprocessor_internet, "./artifacts/internet/preprocessor_internet.pkl")
joblib.dump(model_internet, "./artifacts/internet/model_internet.pkl")

Paramètres : {'learning_rate': np.float64(0.02), 'scale_pos_weight': np.int64(3), 'subsample': np.float64(0.8)}
Score (f1) : 0.5491262486520087


['./artifacts/internet/model_internet.pkl']

### **3.2. Sans Internet**

- Nous poursuivrons avec un `XGBClassifier`, en testant diverses combinaisons de variables.

In [10]:
disc = [
    # "Gender",
    # "Senior Citizen",
    # "Partner",
    "Dependents",
    # "Multiple Lines",
    "Contract",
    "Paperless Billing",
    # "Payment Method",
    # "Is New Client",
]
cont = [
    "Tenure Months",
    # "Monthly Charges",
    # "Total Charges",
    "CLTV",
    "Lat",
    "Long",
    # "Monthly Charges Group",
]

X_no_internet = df_no_internet.drop(columns="Churn Value")
y_no_internet = df_no_internet["Churn Value"]
X_train_no_internet, X_test_no_internet, y_train_no_internet, y_test_no_internet = (
    train_test_split(
        X_no_internet,
        y_no_internet,
        test_size=0.2,
        random_state=42,
        stratify=y_no_internet,
    )
)

preprocessor_no_internet = ColumnTransformer(
    transformers=[
        ("disc", OneHotEncoder(drop="first", handle_unknown="ignore"), disc),
        ("cont", MinMaxScaler(), cont),
    ],
    remainder="drop",
)

X_train_no_internet_preprocessed = preprocessor_no_internet.fit_transform(
    X_train_no_internet
)

clf_no_internet = XGBClassifier(random_state=42)
print(
    f"XGBClassifier : {cross_val_score(clf_no_internet, X_train_no_internet_preprocessed, y_train_no_internet, scoring='recall', cv=5).mean():.4f}"
)

XGBClassifier : 0.0400


- Pour les abonnés sans Internet, le modèle final ne prendra en entrée que 7 variables : `Dependents`, `Contract`, `Paperless Billing`, `Tenure Months`, `CLTV`, `Lat` et `Long`.

In [11]:
model_no_internet = GridSearchCV(clf_no_internet, param_grid, scoring="f1", cv=5)
model_no_internet.fit(X_train_no_internet_preprocessed, y_train_no_internet)

print("Paramètres :", model_no_internet.best_params_)
print("Score (f1) :", model_no_internet.best_score_)

# Exportation du pipeline de preprocessing et du modèle entraînés
joblib.dump(
    preprocessor_no_internet, "./artifacts/no_internet/preprocessor_no_internet.pkl"
)
joblib.dump(model_no_internet, "./artifacts/no_internet/model_no_internet.pkl")

Paramètres : {'learning_rate': np.float64(0.04), 'scale_pos_weight': np.int64(6), 'subsample': np.float64(0.8)}
Score (f1) : 0.09258204334365325


['./artifacts/no_internet/model_no_internet.pkl']

### **3.3. Performances globabes**

In [12]:
# Evaluation sur le test set (Internet)
X_test_internet_preprocessed = preprocessor_internet.transform(X_test_internet)
y_pred_internet = model_internet.predict(X_test_internet_preprocessed)
print(classification_report(y_true=y_test_internet, y_pred=y_pred_internet))

              precision    recall  f1-score   support

           0       0.92      0.84      0.88       732
           1       0.52      0.73      0.61       182

    accuracy                           0.81       914
   macro avg       0.72      0.78      0.74       914
weighted avg       0.84      0.81      0.82       914



In [13]:
# Evaluation sur le test set (No Internet)
X_test_no_internet_preprocessed = preprocessor_no_internet.transform(X_test_no_internet)
y_pred_no_internet = model_no_internet.predict(X_test_no_internet_preprocessed)
print(classification_report(y_true=y_test_no_internet, y_pred=y_pred_no_internet))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       276
           1       0.42      0.38      0.40        13

    accuracy                           0.95       289
   macro avg       0.69      0.68      0.69       289
weighted avg       0.95      0.95      0.95       289



In [14]:
# Evaluation sur le test set (All : modèles spécialisés)
y_true = pd.concat([y_test_internet, y_test_no_internet])
y_pred = np.concat([y_pred_internet, y_pred_no_internet])
print(classification_report(y_true=y_true, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.94      0.87      0.90      1008
           1       0.52      0.70      0.60       195

    accuracy                           0.85      1203
   macro avg       0.73      0.79      0.75      1203
weighted avg       0.87      0.85      0.86      1203



### **3.4. Et si on avait entraîné qu'un seul modèle ?**

In [15]:
disc = [
    # "Gender",
    "Senior Citizen",
    "Partner",
    "Dependents",
    # "Phone Service",
    # "Multiple Lines",
    # "Internet Service",
    "Online Security",
    "Online Backup",
    "Device Protection",
    "Tech Support",
    "Streaming TV",
    "Streaming Movies",
    # "Contract",
    "Paperless Billing",
    # "Payment Method",
    # "Is New Client",
]
cont = [
    "Tenure Months",
    "Monthly Charges",
    "Total Charges",
    "CLTV",
    "Lat",
    "Long",
    # "Monthly Charges Group",  d'office, nous n'allons utiliser ni les clusters
    # "Services Count",         ni le nombre de services Internet dans ce cas
]


X_train_all = pd.concat([X_train_internet, X_train_no_internet])
X_test_all = pd.concat([X_test_internet, X_test_no_internet])
y_train_all = pd.concat([y_train_internet, y_train_no_internet])
y_test_all = pd.concat([y_test_internet, y_test_no_internet])

preprocessor_all = ColumnTransformer(
    transformers=[
        ("disc", OneHotEncoder(drop="first", handle_unknown="ignore"), disc),
        ("cont", MinMaxScaler(), cont),
    ],
    remainder="drop",
)

X_train_all_preprocessed = preprocessor_all.fit_transform(X_train_all)

clf_all = XGBClassifier(random_state=42)
print(
    f"XGBClassifier : {cross_val_score(clf_all, X_train_all_preprocessed, y_train_all, scoring='recall', cv=5).mean():.4f}"
)

XGBClassifier : 0.3192


In [16]:
model_all = GridSearchCV(clf_all, param_grid, scoring="f1", cv=5)
model_all.fit(X_train_all_preprocessed, y_train_all)

print("Paramètres :", model_all.best_params_)
print("Score (f1) :", model_all.best_score_)

# Exportation du pipeline de preprocessing et du modèle entraînés
joblib.dump(preprocessor_all, "./artifacts/all/preprocessor_all.pkl")
joblib.dump(model_all, "./artifacts/all/model_all.pkl")

Paramètres : {'learning_rate': np.float64(0.02), 'scale_pos_weight': np.int64(4), 'subsample': np.float64(0.8)}
Score (f1) : 0.5223340926174663


['./artifacts/all/model_all.pkl']

In [17]:
# Evaluation sur le test set (All : modèle unique)
X_test_all_preprocessed = preprocessor_all.transform(X_test_all)
y_pred_all = model_all.predict(X_test_all_preprocessed)
print(classification_report(y_true=y_test_all, y_pred=y_pred_all))

              precision    recall  f1-score   support

           0       0.94      0.84      0.89      1008
           1       0.46      0.72      0.57       195

    accuracy                           0.82      1203
   macro avg       0.70      0.78      0.73      1203
weighted avg       0.86      0.82      0.83      1203



**Récapitulatif des performances des différents modèles**

| Métrique  | Avec Internet | Sans Internet | Modèle Spécialisé | Modèle Unique |
|-----------|--------------:|--------------:|------------------:|--------------:|
| Accuracy  |        0.8140 |        0.9481 |        **0.8462** |        0.8196 |
| Précision |        0.5238 |        0.4167 |        **0.5189** |        0.4638 |
| Rappel    |        0.7253 |        0.3846 |            0.7026 |    **0.7231** |
| Score f1  |        0.6083 |        0.4000 |        **0.5969** |        0.5651 |

- Notre modèle spécialisé est légèrement meilleur que le modèle unique avec une accuracy à 85%, une précision à 52%, un rappel à 70% et un score f1 à 60%.
- Cela signifie que nous pourrons anticiper 7 départs sur 10, mais également que 1 fois sur 2, nous aurons à faire à des fausses alertes.
- Dans un contexte réel, les politiques de rétention client impliquant des coûts, nous pourrons, après concertation avec les équipes métier, ajuster le seuil de décision afin de trouver le meilleur compromis entre la réduction des fausses alertes et l'efficacité de la prédiction des résiliations.
- Il est également possible d'adopter une stratégie différente selon les probabilités prédites et d'autres critères importants tels que la valeur client, l'ancienneté, etc.
- Aussi, du fait de la rareté des résiliations chez les clients sans Internet, le modèle a davantage de difficultés sur ce segment.
- Pour gagner en performances, on pourrait donc par exemple envisager, dans la mesure du possible, de rajouter davantage d'abonnés appartenant à cette catégorie et ayant résilié leur contrat afin de permettre au modèle de mieux cerner leurs caractéristiques.